In [11]:
import os
import glob
import pickle
import pdb
import ray
import wfdb
import csv
import pandas as pd
from scipy import signal

In [3]:
def read_dat_files(directory):
    dat_files = glob.glob(os.path.join(directory, '**', '*.dat'), recursive=True)
    files = {}
    
    for file_path in dat_files:
        if file_path.split('/')[-1].split('_')[1] == 'ECG.dat':
            record_name = os.path.splitext(file_path)[0]
            record = wfdb.rdrecord(record_name)
            content = record.p_signal[:, 0]
            # pdb.set_trace()
            path = file_path.split('/')[-1].split('_')[0]
            files.update({path:content})
    return files    

# Example usage:
directory = '/home/insightlab/Documents/ECG Quality/brno-university-of-technology-ecg-quality-database-but-qdb-1.0.0'
files = read_dat_files(directory)

In [4]:
def read_ann(directory):
    dat_files = glob.glob(os.path.join(directory, '**', '*.csv'), recursive=True)
    files = pd.DataFrame(columns=['record','ini_cons', 'end_cons', 'label_cons'])
    
    for file_path in dat_files:
        path = file_path.split('/')[-1].split('_')[0]
        if path != 'subject-info.csv':
            df = pd.read_csv(file_path, header=None)
            df.dropna(inplace=True)
            df = df[df.columns[-3:]]
            df.columns = ['ini_cons', 'end_cons', 'label_cons']
            df.drop(df[df['label_cons'] == 0].index, inplace = True)
            df['record'] = path
            files =  pd.concat([files, df], ignore_index=True)
    return files    

# Example usage:
directory = '/home/insightlab/Documents/ECG Quality/brno-university-of-technology-ecg-quality-database-but-qdb-1.0.0'
annotations = read_ann(directory)

/tmp/ipykernel_72056/3140899762.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  files =  pd.concat([files, df], ignore_index=True)


In [13]:
sf = 1000
new_sf = 125

with open('windows_data.pkl', 'ab') as pickle_file:  # 'ab' mode for appending in binary
    for path, sgn in files.items():
        df = annotations[annotations['record'] == path]
        
        for index, row in df.iterrows():
            k = int(row['ini_cons'])
    
            while k + 10 * sf < int(row['end_cons']):
                ini = k
                k += 10 * sf
                resampled_signal = signal.resample(sgn[ini:k], new_sf*10)
                
                record = {
                    'record': path,
                    'window': resampled_signal.tolist(),
                    'label': row['label_cons']
                }
                
                pickle.dump(record, pickle_file)

print("Windows data saved to windows_data.pkl")

Windows data saved to windows_data.pkl
